In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
import pandas as pd
from utils import *
%pylab inline --no-import-all

Populating the interactive namespace from numpy and matplotlib


In [3]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.io #Used to load the OCTAVE *.mat files
import scipy.misc #Used to show matrix as an image
import matplotlib.cm as cm #Used to display images in a specific colormap
import random #To pick random images to display
import scipy.optimize #fmin_cg to train neural network
import itertools
from scipy.special import expit #Vectorized sigmoid function

In [4]:
test_df = data_df[data_df.date < "2015-01-01"]
data_df = data_df[(data_df.date > "2015-01-01") & (data_df.date < "2016-01-01")]

In [26]:
data_df = test_df

In [27]:
df = pd.DataFrame(index=data_df.index)

In [28]:
columns = ["yz0","yz1","minute","minute2","isnew","opentop0","opentop1","opentop2","reachtop1","top0","top1","top2","foot0","foot1","foot2","jump1","jump2","speedup1","speedup2","small_volume"]

In [29]:
for column in columns:
    df[column] = eval(column).astype(int)

In [30]:
# df["volume_ratio"] = data_df["minute_volume"] / data_df["volume1"
len(data_df)

13185

In [31]:
change_se = data_df["change"].copy()
change_se[change_se > 1.0] = 1
change_se[change_se < 1.0] = 0 

In [32]:
X = df.as_matrix()
y = change_se.as_matrix()
#Insert a column of 1's to X as usual
X = np.insert(X,0,1,axis=1)
print "'y' shape: %s. Unique elements in y: %s"%(y.shape,np.unique(y))
print "'X' shape: %s. X[0] shape: %s"%(X.shape,X[0].shape)

'y' shape: (13185,). Unique elements in y: [ 0.  1.]
'X' shape: (13185, 21). X[0] shape: (21,)


In [12]:
# These are some global variables I'm suing to ensure the sizes
# of various matrices are correct
#these are NOT including bias nits
input_layer_size = 20
hidden_layer_size = 7
output_layer_size = 2 
n_training_samples = X.shape[0]

In [13]:
#Some utility functions. There are lot of flattening and
#reshaping of theta matrices, the input X matrix, etc...
#Nicely shaped matrices make the linear algebra easier when developing,
#but the minimization routine (fmin_cg) requires that all inputs

def flattenParams(thetas_list):
    """
    Hand this function a list of theta matrices, and it will flatten it
    into one long (n,1) shaped numpy array
    """
    flattened_list = [ mytheta.flatten() for mytheta in thetas_list ]
    combined = list(itertools.chain.from_iterable(flattened_list))
    assert len(combined) == (input_layer_size+1)*hidden_layer_size + \
                            (hidden_layer_size+1)*output_layer_size
    return np.array(combined).reshape((len(combined),1))

def reshapeParams(flattened_array):
    theta1 = flattened_array[:(input_layer_size+1)*hidden_layer_size] \
            .reshape((hidden_layer_size,input_layer_size+1))
    theta2 = flattened_array[(input_layer_size+1)*hidden_layer_size:] \
            .reshape((output_layer_size,hidden_layer_size+1))
    
    return [ theta1, theta2 ]

def flattenX(myX):
    return np.array(myX.flatten()).reshape((n_training_samples*(input_layer_size+1),1))

def reshapeX(flattenedX):
    return np.array(flattenedX).reshape((n_training_samples,input_layer_size+1))

In [14]:
def computeCost(mythetas_flattened,myX_flattened,myy,mylambda=0.):
    """
    This function takes in:
        1) a flattened vector of theta parameters (each theta would go from one
           NN layer to the next), the thetas include the bias unit.
        2) the flattened training set matrix X, which contains the bias unit first column
        3) the label vector y, which has one column
    It loops over training points (recommended by the professor, as the linear
    algebra version is "quite complicated") and:
        1) constructs a new "y" vector, with 10 rows and 1 column, 
            with one non-zero entry corresponding to that iteration
        2) computes the cost given that y- vector and that training point
        3) accumulates all of the costs
        4) computes a regularization term (after the loop over training points)
    """
    
    # First unroll the parameters
    mythetas = reshapeParams(mythetas_flattened)
    
    # Now unroll X
    myX = reshapeX(myX_flattened)
    
    #This is what will accumulate the total cost
    total_cost = 0.
    
    m = n_training_samples

    # Loop over the training points (rows in myX, already contain bias unit)
    for irow in xrange(m):
        myrow = myX[irow]
                
        # First compute the hypothesis (this is a (10,1) vector
        # of the hypothesis for each possible y-value)
        # propagateForward returns (zs, activations) for each layer
        # so propagateforward[-1][1] means "activation for -1st (last) layer"
        myhs = propagateForward(myrow,mythetas)[-1][1]

        # Construct a 10x1 "y" vector with all zeros and only one "1" entry
        # note here if the hand-written digit is "0", then that corresponds
        # to a y- vector with 1 in the 10th spot (different from what the
        # homework suggests)
        tmpy  = np.zeros((output_layer_size,1))
        tmpy[int(myy[irow]-1)] = 1
        
        # Compute the cost for this point and y-vector
        mycost = -tmpy.T.dot(np.log(myhs))-(1-tmpy.T).dot(np.log(1-myhs))
     
        # Accumulate the total cost
        total_cost += mycost
  
    # Normalize the total_cost, cast as float
    total_cost = float(total_cost) / m
    
    # Compute the regularization term
    total_reg = 0.
    for mytheta in mythetas:
        total_reg += np.sum(mytheta*mytheta) #element-wise multiplication
    total_reg *= float(mylambda)/(2*m)
        
    return total_cost + total_reg
       

def propagateForward(row,Thetas):
    """
    Function that given a list of Thetas (NOT flattened), propagates the
    row of features forwards, assuming the features ALREADY
    include the bias unit in the input layer, and the 
    Thetas also include the bias unit

    The output is a vector with element [0] for the hidden layer,
    and element [1] for the output layer
        -- Each element is a tuple of (zs, as)
        -- where "zs" and "as" have shape (# of units in that layer, 1)
    
    ***The 'activations' are the same as "h", but this works for many layers
    (hence a vector of thetas, not just one theta)
    Also, "h" is vectorized to do all rows at once...
    this function takes in one row at a time***
    """
   
    features = row
    zs_as_per_layer = []
    for i in xrange(len(Thetas)):  
        Theta = Thetas[i]
        #Theta is (25,401), features are (401, 1)
        #so "z" comes out to be (25, 1)
        #this is one "z" value for each unit in the hidden layer
        #not counting the bias unit
        z = Theta.dot(features).reshape((Theta.shape[0],1))
        a = expit(z)
        zs_as_per_layer.append( (z, a) )
        if i == len(Thetas)-1:
            return np.array(zs_as_per_layer)
        a = np.insert(a,0,1) #Add the bias unit
        features = a

In [15]:
def genRandThetas():
    epsilon_init = 0.12
    theta1_shape = (hidden_layer_size, input_layer_size+1)
    theta2_shape = (output_layer_size, hidden_layer_size+1)
    rand_thetas = [ np.random.rand( *theta1_shape ) * 2 * epsilon_init - epsilon_init, \
                    np.random.rand( *theta2_shape ) * 2 * epsilon_init - epsilon_init]
    return rand_thetas

In [16]:
def sigmoidGradient(z):
    dummy = expit(z)
    return dummy*(1-dummy)

In [17]:
def backPropagate(mythetas_flattened,myX_flattened,myy,mylambda=0.):
    
    # First unroll the parameters
    mythetas = reshapeParams(mythetas_flattened)
    
    # Now unroll X
    myX = reshapeX(myX_flattened)

    #Note: the Delta matrices should include the bias unit
    #The Delta matrices have the same shape as the theta matrices
    Delta1 = np.zeros((hidden_layer_size,input_layer_size+1))
    Delta2 = np.zeros((output_layer_size,hidden_layer_size+1))

    # Loop over the training points (rows in myX, already contain bias unit)
    m = n_training_samples
    for irow in xrange(m):
        myrow = myX[irow]
        a1 = myrow.reshape((input_layer_size+1,1))
        # propagateForward returns (zs, activations) for each layer excluding the input layer
        temp = propagateForward(myrow,mythetas)
        z2 = temp[0][0]
        a2 = temp[0][1]
        z3 = temp[1][0]
        a3 = temp[1][1]
        tmpy = np.zeros((output_layer_size,1))
        tmpy[int(myy[irow]-1)] = 1
        delta3 = a3 - tmpy 
        delta2 = mythetas[1].T[1:,:].dot(delta3)*sigmoidGradient(z2) #remove 0th element
        a2 = np.insert(a2,0,1,axis=0)
        Delta1 += delta2.dot(a1.T) #(25,1)x(1,401) = (25,401) (correct)
        Delta2 += delta3.dot(a2.T) #(10,1)x(1,25) = (10,25) (should be 10,26)
        
    D1 = Delta1/float(m)
    D2 = Delta2/float(m)
    
    #Regularization:
    D1[:,1:] = D1[:,1:] + (float(mylambda)/m)*mythetas[0][:,1:]
    D2[:,1:] = D2[:,1:] + (float(mylambda)/m)*mythetas[1][:,1:]
    
    return flattenParams([D1, D2]).flatten()

In [18]:
#Once you are done, using the loaded set of parameters Theta1 and Theta2,
#and lambda = 1, you should see that the cost is about 0.383770
myThetas = genRandThetas()
print computeCost(flattenParams(myThetas),flattenX(X),y)

1.39804525273


In [19]:
#Actually compute D matrices for the Thetas provided
flattenedD1D2 = backPropagate(flattenParams(myThetas),flattenX(X),y,mylambda=0.1)
D1, D2 = reshapeParams(flattenedD1D2)

In [20]:
def checkGradient(mythetas,myDs,myX,myy,mylambda=0.):
    myeps = 0.0001
    flattened = flattenParams(mythetas) #把backprop计算出来的theta拍平
    flattenedDs = flattenParams(myDs) #把backprop计算出来的D拍平
    myX_flattened = flattenX(myX)
    n_elems = len(flattened) 
    #Pick ten random elements, compute numerical gradient, compare to respective D's
    for i in xrange(10):
        x = int(np.random.rand()*n_elems)
        epsvec = np.zeros((n_elems,1)) #创建一个theta的数组
        epsvec[x] = myeps #epsilon
        cost_high = computeCost(flattened + epsvec,myX_flattened,myy,mylambda) #theta+ = flattened + epsvec
        cost_low  = computeCost(flattened - epsvec,myX_flattened,myy,mylambda)  #theta+ = flattened - epsvec
        mygrad = (cost_high - cost_low) / float(2*myeps)
        print "Element: %d. Numerical Gradient = %f. BackProp Gradient = %f."%(x,mygrad,flattenedDs[x])

In [21]:
checkGradient(myThetas,[D1, D2],X,y)

Element: 71. Numerical Gradient = 0.000141. BackProp Gradient = 0.000142.
Element: 94. Numerical Gradient = -0.000268. BackProp Gradient = -0.000268.
Element: 146. Numerical Gradient = -0.002244. BackProp Gradient = -0.002244.
Element: 54. Numerical Gradient = -0.005950. BackProp Gradient = -0.005950.
Element: 6. Numerical Gradient = 0.000019. BackProp Gradient = 0.000020.


KeyboardInterrupt: 

In [22]:
#Here I will use scipy.optimize.fmin_cg

def trainNN(mylambda=0.):
    """
    Function that generates random initial theta matrices, optimizes them,
    and returns a list of two re-shaped theta matrices
    """

    randomThetas_unrolled = flattenParams(genRandThetas())
    result = scipy.optimize.fmin_cg(computeCost, x0=randomThetas_unrolled, fprime=backPropagate, \
                               args=(flattenX(X),y,mylambda),maxiter=100,disp=True,full_output=True)
    return reshapeParams(result[0])

In [23]:
#Training the NN takes about ~70-80 seconds on my machine
learned_Thetas = trainNN()

         Current function value: 0.787061
         Iterations: 100
         Function evaluations: 215
         Gradient evaluations: 215


In [50]:
def predictNN(row,Thetas):
    """
    Function that takes a row of features, propagates them through the
    NN, and returns the predicted integer that was hand written
    """
    classes = range(0,2)
    output = propagateForward(row,Thetas)
    #-1 means last layer, 1 means "a" instead of "z"
#     print classes,np.argmax(output[-1][1])
    return classes[np.argmax(output[-1][1])] 

def computeAccuracy(myX,myThetas,myy):
    """
    Function that loops over all of the rows in X (all of the handwritten images)
    and predicts what digit is written given the thetas. Check if it's correct, and
    compute an efficiency.
    """
    n_correct, n_total = 0, myX.shape[0]
    for irow in xrange(n_total):
#         print int(predictNN(myX[irow],myThetas))
        if int(predictNN(myX[irow],myThetas)) != int(myy[irow]): 
            n_correct += 1
    print "Training set accuracy: %0.1f%%"%(100*(float(n_correct)/n_total))

In [51]:
computeAccuracy(X,learned_Thetas,y)

Training set accuracy: 81.4%


In [ ]:
#Let's see if I set lambda to 10, if I get the same thing
learned_regularized_Thetas = trainNN(mylambda=10.)

In [ ]:
computeAccuracy(X,learned_regularized_Thetas,y)

In [34]:
y_se = pd.Series(data_df["change"].as_matrix())
y_se.head(3)

0    1.037694
1    0.957447
2    1.017380
dtype: float64

In [65]:
n_correct, n_total = 0,0
net_value = 1.0
for irow in xrange( X.shape[0]):
#         print int(predictNN(myX[irow],myThetas))
#     if int(predictNN(X[irow],learned_Thetas)) != int(y[irow]): 
#         n_correct += 1
#     print int(predictNN(X[irow],learned_Thetas))

    y_value = y_se[irow]
    if int(predictNN(X[irow],learned_Thetas)) == 0:
        n_total += 1
        net_value = net_value * y_value
        print y_value
        if y_value >= 1.0:
            n_correct += 1
#     else:
#         if y_value < 1.0:
#             n_correct += 1
print n_correct,n_total,net_value
print "Training set accuracy: %0.1f%%"%(100*(float(n_correct)/n_total))

1.0376940133
0.957446808511
1.01737967914
0.965009208103
0.974126778784
0.980694980695
1.03092783505
0.99504950495
1.04451038576
1.04851752022
1.00490196078
1.01103752759
1.02996254682
1.03661327231
1.02917771883
0.979462875197
1.04596834966
1.00584795322
0.972826086957
1.03858267717
0.988165680473
1.0306345733
1.04049493813
1.01022494888
0.97506234414
1.0612745098
1.0
1.03012912482
1.02044609665
1.20902255639
1.03582395087
1.00506329114
1.00814663951
0.966850828729
1.09322033898
1.06722689076
1.03312101911
0.962962962963
1.03359173127
1.00638569604
1.00470588235
0.986842105263
1.00159172304
0.991363355953
0.98275862069
0.989804587935
0.98768115942
1.00141442716
1.03201754386
1.02319236016
1.04481792717
0.986928104575
0.955367913148
1.10009354537
1.03571428571
0.999261447563
1.02256944444
1.08202443281
1.0
1.09970958374
1.04376012966
1.01694915254
1.09979906229
1.03747870528
1.01818181818
1.01239669421
0.986093552465
0.998850574713
1.01253918495
0.967948717949
1.05700325733
1.001479289

In [35]:
X.shape[0]

13185